In [6]:
from functools import wraps
import pandas as pd
from openpyxl import load_workbook
from openpyxl import Workbook
import os
import timeit


def FindCol(df, tgt_string, replace_string = None):
    tgt_cols = [col for col in df.columns if tgt_string in col]
    df = df[tgt_cols]

    if replace_string != None:
        df = df.rename(columns=lambda s: s.replace(tgt_string, replace_string))
        
    return df

def SaveExcel(df, name):
    file_path = "P:\\Product Specialists\\Jonathan\\Seismic\\TimeSeries\\" + name + ".xlsx"
    
    workbook = Workbook()
    sheet = workbook.active
    sheet.title = name
    workbook.save(file_path)

    file_path = "P:\\Product Specialists\\Jonathan\\Seismic\\TimeSeries\\" + name + ".xlsx"
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        df.to_excel(writer, sheet_name = name, index = False)
    
def ColumnOrder(df):    
    try:
        first_column = df.pop("Code") 
        df.insert(0, "Code", first_column) 
    except:
        pass
    
    for (index, colname) in enumerate(df):
        if colname != "Code" and colname != "Date":
            try:
                df[colname] = df[colname].astype("float64")
            except:
                df[colname] = df[colname]
    return df

def GetMD(file_path):
    df = pd.read_excel(file_path,sheet_name = "H Risk Indicators", index_col=0)
    df_MD = FindCol(df, "_Sensibilité", " Modified Duration")
    df_MD = df_MD.fillna(0).astype('float64')
    df_MD["Code"]=ticker
    return df_MD

def GetDTS(file_path):
    df = pd.read_excel(file_path,sheet_name = "H Risk Indicators", index_col=0)
    df_DTS =  FindCol(df, "_SPS", " DTS")
    df_DTS["Code"]=ticker
    return df_DTS 
    
def GetFX(file_path):
    df = pd.read_excel(file_path,sheet_name = "H Breakdown by Currency", index_col=0)
    df_FX = FindCol(df, "Wght% (PTF-REF)_", "")
    df_FX["Code"]=ticker
    return df_FX

def GetTE(file_path):
    df = pd.read_excel(file_path,sheet_name = "H Tracking Error Breakdown", index_col=0)
    df_TE = FindCol(df, "Risk")[["Global Risk","Forex Risk","IR Market Risk","- Credit Emg Risk","- Credit Agg Risk","Inflation Risk"]]
    df_TE = df_TE.rename(columns={"- Credit Emg Risk": "Emerging Markets Risk", "- Credit Agg Risk": "Credit Risk"})
    for i in range(len(df_TE)):
        df_TE.iloc[i] = df_TE.iloc[i].str.rstrip('%').astype('float') / 100.0        

        TE_Addup = df_TE.iloc[i,1] + df_TE.iloc[i,2] + df_TE.iloc[i,3] + df_TE.iloc[i,4] + df_TE.iloc[i,5]

        for j in range(len(df_TE.columns)-1):         
            df_TE.iloc[i,j+1] = df_TE.iloc[i,j+1]/TE_Addup*df_TE.iloc[i,0] 

    df_TE["Code"]=ticker    
    return df_TE

def GetRating(file_path):
    df = pd.read_excel(file_path,sheet_name = "H Breakdown by Rating", index_col=0)
    #df_Rating_P = FindCol(df, "Expo weight (PTF)_", "Portfolio ")/100
    try:
        #df_Rating_P = FindCol(df, "Expo weight (PTF)_", "Portfolio ")/100
        """
        df_Rating_P = FindCol(df, "Wght% (PTF)_", "Portfolio ")/100
        df_Rating_B= FindCol(df, "Wght% (REF)_", "Benchmark ")/100
        """        
        df_Rating_P = FindCol(df, "Wght% (PTF)_", "Portfolio ")
        df_Rating_B= FindCol(df, "Wght% (REF)_", "Benchmark ")
    except:
        #df_Rating_P = FindCol(df, "Expo weight (PTF)_", "Portfolio ")/100
        df_Rating_P = FindCol(df, "Wght% (PTF)_", "Portfolio ")
        df_Rating_B= FindCol(df, "Wght% (REF)_", "Benchmark ")

    df_Rating = pd.concat([df_Rating_P, df_Rating_B], axis=1)  
    df_Rating["Code"]=ticker
    return df_Rating

def GetRiskYC(file_path):
    try:
        df = pd.read_excel(file_path,sheet_name = "H Risk per YC", index_col=0)
        df_Risk_YC_P = FindCol(df, "MD c. (PTF)_", "")
        df_Risk_YC_B= FindCol(df, "MD c. (REF)_", "")

        df_Risk_YC = df_Risk_YC_P.subtract(df_Risk_YC_B, fill_value=0, axis=1)
        df_Risk_YC["Code"]=ticker
    except:
        try:
            df = pd.read_excel(file_path,sheet_name = "H Risk per YC", index_col=0)
            df_Risk_YC_P = FindCol(df, "MD c. (PTF-REF)_", "")
            df_Risk_YC["Code"]=ticker
        except:
            pass
    return df_Risk_YC

def GetSecotr(file_path):
    df = pd.read_excel(file_path,sheet_name = "H Sector DTS", index_col=0)
    
    try:
        df_Financials_expo = FindCol(df, "Expo weight (PTF)_Financials", "Financials Expo")/100
        df_Financials_dts = FindCol(df, "DTS c. (PTF)_Financials", "Financials DTS")
    except:
        pass
    
    try:
        df_Industrials_expo = FindCol(df, "Expo weight (PTF)_Industrials", "non Financials Expo")/100
        df_Industrials_dts = FindCol(df, "DTS c. (PTF)_Industrials", "non Financials DTS")
    except:
        pass
    
    try:
        df_Utilities_expo = FindCol(df, "Expo weight (PTF)_Utilities", "non Financials Expo")/100
        df_Utilities_dts = FindCol(df, "DTS c. (PTF)_Utilities", "non Financials DTS")
    except:
        pass
    
    try:
        df_Securitized_expo = FindCol(df, "Expo weight (PTF)_Securitized", "non Financials Expo")/100
        df_Securitized_dts = FindCol(df, "DTS c. (PTF)_Securitized", "non Financials DTS")
    except:
        pass
    
    try:
        df_nonFinancials_expo = df_Industrials_expo.add(df_Utilities_expo.add(df_Securitized_expo))
        df_nonFinancials_dts = df_Industrials_dts.add(df_Utilities_dts.add(df_Securitized_dts))
    except:
        pass
    
    try:
        df_expo = df_Financials_expo.add(df_nonFinancials_expo, fill_value = 0)
        df_dts = df_Financials_dts.add(df_nonFinancials_dts, fill_value = 0)
        df_expo["Code"]=ticker
        df_dts["Code"]=ticker
    except:
        pass
    
    return df_expo, df_dts
    
    
@TryExcept
def SetIndex(df_main, df):    
    df_main = pd.concat([df_main, df.reset_index().rename(columns={"index" : "Date"})])
    return df_main
    

    
print("Start exporting...")
        
df_MD_main = pd.DataFrame()
df_DTS_main = pd.DataFrame()
df_FX_main = pd.DataFrame()
df_TE_main = pd.DataFrame()
df_Rating_main = pd.DataFrame()
df_Risk_YC_main = pd.DataFrame()
df_expo_main = pd.DataFrame()
df_dts_main = pd.DataFrame()        
        
tickers = ["X2947","MULTI","UNOB","PF60091","PHMDE","E2525","D1945L",
           "PF81742","PF87396","PF56896","PF40733", "PF82494","PF56190",
           "PF56699","PF49055", "M22285","PF58925", "5742995","PF37614",
           "PF69405", "M27828","PF88902","S105195","PF61685","PF48158",
           "PF103973", "PF45040","2SICFAL", "C2525R", "D2525","I40",
           "PF90548","PF71258","GIFGL","X2975","PF88017","PF37442",
           "X20400","PF103995","PF55902","PF55645","D1945C", "D318M",
           "X68085", "PF58175","PF82728","PF96625","PF92671","S2525",
           "L837","PF56415","X1144","PF71798","PF47190","M29448","PF92166"]
tickers = ["GIFEC"]
insurance = ["PF82700"]

for ticker in tickers:
    file_path = "P:\\Product Specialists\\Tools\\Python Tools\\db_trial\\" + ticker + "_Slides_Data.xlsx"
    
    df_MD = GetMD(file_path)
    df_DTS = GetDTS(file_path)
    df_FX = GetFX(file_path)
    df_TE = GetTE(file_path)
    df_Rating = GetRating(file_path)
    df_Risk_YC = GetRiskYC(file_path)
    df_expo, df_dts = GetSector(file_path)
   
    df_MD_main = SetIndex(df_MD_main, df_MD)
    df_DTS_main = SetIndex(df_DTS_main, df_DTS)
    df_FX_main = SetIndex(df_FX_main, df_FX)
    df_TE_main = SetIndex(df_TE_main, df_TE)
    df_Rating_main = SetIndex(df_Rating_main, df_Rating)
    df_Risk_YC_main = SetIndex(df_Risk_YC_main, df_Risk_YC)
    df_expo_main = SetIndex(df_expo_main, df_expo)
    df_dts_main = SetIndex(df_dts_main, df_dts)
    

    
    #df_MD_main = pd.concat([df_MD_main, df_MD.reset_index().rename(columns={"index" : "Date"})])
    #df_DTS_main = pd.concat([df_DTS_main,df_DTS.reset_index().rename(columns={"index" : "Date"})])
    #df_FX_main = pd.concat([df_FX_main,df_FX.reset_index().rename(columns={"index" : "Date"})])
    #df_TE_main = pd.concat([df_TE_main,df_TE.reset_index().rename(columns={"index" : "Date"})])
    #df_Rating_main = pd.concat([df_Rating_main,df_Rating.reset_index().rename(columns={"index" : "Date"})])           
    #df_Risk_YC_main = pd.concat([df_Risk_YC_main,df_Risk_YC.reset_index().rename(columns={"index" : "Date"})])
    #df_expo_main = pd.concat([df_expo_main,df_expo.reset_index().rename(columns={"index" : "Date"})])
    #df_dts_main = pd.concat([df_dts_main,df_dts.reset_index().rename(columns={"index" : "Date"})])

try:
    """
    df_Rating_main = df_Rating_main.loc[:, ["Date",
                                            "Portfolio AAA","Portfolio AA","Portfolio A",
                                            "Portfolio BBB","Portfolio BB","Portfolio B",
                                            "Portfolio CCC","Portfolio NR","Portfolio Unknown",
                                            "Benchmark AAA","Benchmark AA","Benchmark A",
                                            "Benchmark BBB","Benchmark BB","Benchmark B",
                                            "Benchmark CCC","Benchmark NR","Benchmark Unknown","Code"]]
    
    """
except:
    pass

df_MD_main = df_MD_main.loc[:, ["Date","Code","PTF Modified Duration","REF Modified Duration"]]
df_DTS_main = df_DTS_main.loc[:, ["Date","Code","PTF DTS","REF DTS"]]

print("Saving files...")    

SaveExcel(ColumnOrder(df_MD_main), "H Modified Duration")
SaveExcel(ColumnOrder(df_DTS_main), "H DTS")
SaveExcel(ColumnOrder(df_FX_main), "H FX")
SaveExcel(df_TE_main, "H Tracking Error")
SaveExcel(df_Rating_main, "H Rating")

SaveExcel(ColumnOrder(df_Risk_YC_main), "H Risk per YC")
SaveExcel(ColumnOrder(df_expo_main), "H Secotr Expo")
SaveExcel(ColumnOrder(df_dts_main), "H Sector DTS")    

print("\nComplete")

Start exporting...


KeyError: "['Global Risk'] not in index"

In [5]:
from functools import wraps
import pandas as pd
from openpyxl import load_workbook
from openpyxl import Workbook
import os
import timeit

def TryExcept(func):
    def wrap(*args, **kargs):
        try:
            func(*args, **kargs)
        except:
            pass
        
def works(start, stop):
    total = 0
    for i in range(start, stop):
        total += i
    print("total:", total)
    
works(1, )

TypeError: works() missing 1 required positional argument: 'stop'

In [ ]:
    # get fx
    try:
        df = pd.read_excel(file_path,sheet_name = "H Breakdown by Currency", index_col=0)
        df_FX = FindCol(df, "Wght% (PTF-REF)_", "")
        df_FX["Code"]=ticker
    except:
        pass
    
    # get TE
    try:
        df = pd.read_excel(file_path,sheet_name = "H Tracking Error Breakdown", index_col=0)
        df_TE = FindCol(df, "Risk")[["Global Risk","Forex Risk","IR Market Risk","- Credit Emg Risk","- Credit Agg Risk","Inflation Risk"]]
        for i in range(len(df_TE)):
            df_TE.iloc[i] = df_TE.iloc[i].str.rstrip('%').astype('float') / 100.0
            for j in range(len(df_TE.columns)-1):
                TE_Addup = df_TE.iloc[i,1] + df_TE.iloc[i,2] + df_TE.iloc[i,3] + df_TE.iloc[i,4] + df_TE.iloc[i,5]
                df_TE.iloc[i,j+1] = df_TE.iloc[i,j+1]/TE_Addup        
        df_TE["Code"]=ticker
    except:
        pass


    # get rating
    try:
        df = pd.read_excel(file_path,sheet_name = "H Breakdown by Rating", index_col=0)
        df_Rating_P = int(FindCol(df, "Expo weight (PTF)_", "Portfolio "))/100
        df_Rating_B= int(FindCol(df, "Wght% (REF)_", "Benchmark "))/100

        df_Rating = pd.concat([df_Rating_P, df_Rating_B], axis=1)
        df_Rating["Code"]=ticker
    except:
        pass
    
    # get md for each ccy
    try:
        df = pd.read_excel(file_path,sheet_name = "H Risk per YC", index_col=0)
        df_Risk_YC_P = FindCol(df, "MD c. (PTF)_", "Portfolio ")
        df_Risk_YC_B= FindCol(df, "MD c. (REF)_", "Benchmark ")

        df_Risk_YC = pd.concat([df_Risk_YC_P, df_Risk_YC_B], axis=1)
        df_Risk_YC["Code"]=ticker
    except:
        pass
    
    # get expo and dts for each sector

    df = pd.read_excel(file_path,sheet_name = "H Sector DTS", index_col=0)
    
    try:
        df_Financials_expo = FindCol(df, "Expo weight (PTF)_Financials", "Financials Expo")/100
        df_Financials_dts = FindCol(df, "DTS c. (PTF)_Financials", "Financials DTS")
    except:
        pass
    
    try:
        df_Industrials_expo = FindCol(df, "Expo weight (PTF)_Industrials", "non Financials Expo")/100
        df_Industrials_dts = FindCol(df, "DTS c. (PTF)_Industrials", "non Financials DTS")
    except:
        pass
    
    try:
        df_Utilities_expo = FindCol(df, "Expo weight (PTF)_Utilities", "non Financials Expo")/100
        df_Utilities_dts = FindCol(df, "DTS c. (PTF)_Utilities", "non Financials DTS")
    except:
        pass
    
    try:
        df_Securitized_expo = FindCol(df, "Expo weight (PTF)_Securitized", "non Financials Expo")/100
        df_Securitized_dts = FindCol(df, "DTS c. (PTF)_Securitized", "non Financials DTS")
    except:
        pass
    
    try:
        df_nonFinancials_expo = df_Industrials_expo.add(df_Utilities_expo.add(df_Securitized_expo))
        df_nonFinancials_dts = df_Industrials_dts.add(df_Utilities_dts.add(df_Securitized_dts))
    except:
        pass
    
    try:
        df_expo = df_Financials_expo.add(df_nonFinancials_expo, fill_value = 0)
        df_dts = df_Financials_dts.add(df_nonFinancials_dts, fill_value = 0)
        df_expo["Code"]=ticker
        df_dts["Code"]=ticker
    except:
        pass
    
    try:
        df_MD_main = pd.concat([df_MD_main, df_MD])
        df_DTS_main = pd.concat([df_DTS_main,df_DTS])
        df_FX_main = pd.concat([df_FX_main,df_FX])
        df_TE_main = pd.concat([df_TE_main,df_TE])
        df_Rating_main = pd.concat([df_Rating_main,df_Rating])
        df_Risk_YC_main = pd.concat([df_Risk_YC_main,df_Risk_YC])
        df_expo_main = pd.concat([df_expo_main,df_expo])
        df_dts_main = pd.concat([df_dts_main,df_dts])
    except:
        pass
    print("    " + ticker + " has been added.")


print("Saving files...")    
SaveExcel(df_MD_main, "H Modified Duration")
SaveExcel(df_DTS_main, "H DTS")
SaveExcel(df_FX_main, "H FX")
SaveExcel(df_TE_main, "H Tracking Error")
SaveExcel(df_Rating_main, "H Rating")
SaveExcel(df_Risk_YC_main, "H Risk per YC")
SaveExcel(df_expo_main, "H Secotr Expo")
SaveExcel(df_dts_main, "H Sector DTS")    
print("\nComplete")

In [110]:
sheetname = "H Risk Indicators AF"
df = pd.read_excel(file_path,sheet_name = sheetname, index_col=0)

df_MD = FindCol(df, "_Sensibilité", " Modified Duration")
df_MD.head()

,PTF Modified Duration,REF Modified Duration
2023-09-30,7.03,6.72
2023-08-31,6.29,6.93
2023-07-31,6.46,6.99
2023-06-30,6.53,7.09
2023-05-31,5.99,7.10


In [111]:
sheetname = "H Risk Indicators"
df = pd.read_excel(file_path,sheet_name = sheetname, index_col=0)

df_MD = FindCol(df, "_Sensibilité", " Modified Duration")

df_MD.head()

,PTF Modified Duration,REF Modified Duration,Code
2023-09-30,7.03,6.72,GIFGL
2023-08-31,6.29,6.93,GIFGL
2023-07-31,6.46,6.99,GIFGL
2023-06-30,6.49,7.09,GIFGL
2023-05-31,5.99,7.10,GIFGL


In [75]:
sheetname = "H Risk Indicators"
df = pd.read_excel(file_path,sheet_name = sheetname, index_col=0)

df_DTS = FindCol(df, "_SPS", " DTS")
df_DTS.head()

,PTF DTS,REF DTS
2023-09-30,3.94,1.12
2023-08-31,3.79,1.09
2023-07-31,3.80,1.08


In [34]:
sheetname = "H Breakdown by Currency"
df = pd.read_excel(file_path,sheet_name = sheetname, index_col=0)

df_FX = FindCol(df, "Wght% (PTF-REF)_", "")
df_FX.head()

,NOK,MXN,EUR,KRW,COP,CLP,AUD,HUF,PLN,CNH,SEK,GBP,TWD,ILS,NZD,USD,CHF,JPY,CAD,TRY
2023-09-30,3.3,2.3,1.0,0.9,0.7,0.7,0.6,0.4,-0.3,-0.3,-0.8,-0.8,-0.9,-1.0,-1.1,-2.0,-2.7,0.0,0.0,0.0
2023-08-31,3.1,2.3,-0.2,0.9,0.7,0.7,0.6,0.4,-0.3,-0.3,-0.7,-0.9,-0.9,-1.0,-1.0,-0.7,-2.6,0.1,-0.1,0.0
2023-07-31,3.0,2.2,-0.9,0.9,0.7,0.7,0.6,0.4,0.5,-0.3,-1.2,-0.9,-0.9,-1.0,-1.0,-0.3,-2.6,0.3,-0.1,0.0
2023-06-30,1.9,2.2,-0.7,0.9,0.7,0.7,0.6,0.4,0.5,-0.3,-0.3,-1.0,-0.9,-1.0,-1.0,-0.4,-2.6,0.5,-0.1,0.0
2023-05-31,2.1,2.1,-1.4,1.1,0.8,0.9,0.3,0.5,0.6,-0.3,0.4,-1.7,-1.2,-1.3,-1.3,0.2,-2.7,0.6,0.3,0.0


In [56]:
sheetname = "H Tracking Error Breakdown"
df = pd.read_excel(file_path,sheet_name = sheetname, index_col=0)

df_TE = FindCol(df, "Risk")[["Global Risk","Forex Risk","IR Market Risk","- Credit Emg Risk","- Credit Agg Risk","Inflation Risk"]]

for i in range(len(df_TE)):
    df_TE.iloc[i] = df_TE.iloc[i].str.rstrip('%').astype('float') / 100.0
    for j in range(len(df_TE.columns)-1):
        TE_Addup = df_TE.iloc[i,1] + df_TE.iloc[i,2] + df_TE.iloc[i,3] + df_TE.iloc[i,4] + df_TE.iloc[i,5]
        df_TE.iloc[i,j+1] = df_TE.iloc[i,j+1]/TE_Addup

df_TE.head()

,Global Risk,Forex Risk,IR Market Risk,- Credit Emg Risk,- Credit Agg Risk,Inflation Risk
2019-12-31,0.0241,0.257764,0.052545,0.006575,0.01235,0.00903
2020-02-29,0.0237,0.237805,0.054032,0.006609,0.01725,0.010965
2020-03-31,0.0288,0.225581,0.045194,0.011629,0.040918,0.017921
2020-05-31,0.0214,0.178571,0.053613,0.012644,0.049138,0.016066
2020-06-30,0.0228,0.183099,0.04479,0.013339,0.044284,0.016534


In [69]:
sheetname = "H Breakdown by Rating"
df = pd.read_excel(file_path,sheet_name = sheetname, index_col=0)

df_Rating_P = FindCol(df, "Expo weight (PTF)_", "Portfolio ")/100
df_Rating_B= FindCol(df, "Wght% (REF)_", "Benchmark ")/100

df_Rating = pd.concat([df_Rating_P, df_Rating_B], axis=1)
df_Rating.head()


,Portfolio Total,Portfolio AAA,Portfolio AA,Portfolio A,Portfolio BBB,Portfolio B,Portfolio NR,Portfolio Unknown,Portfolio BB,Benchmark Total,Benchmark AAA,Benchmark AA,Benchmark A,Benchmark BBB,Benchmark B,Benchmark NR,Benchmark Unknown,Benchmark BB
2023-09-30,0.832,0.258,0.251,0.215,0.109,0.000,0.0,0.0,0.0,1.0,0.597,0.132,0.216,0.054,0.0,0.000,0.0,0.0
2023-08-31,0.000,0.280,0.210,0.212,0.115,0.000,0.0,0.0,0.0,0.0,0.593,0.134,0.217,0.056,0.0,0.000,0.0,0.0
2023-07-31,0.000,0.339,0.209,0.212,0.116,0.000,0.0,0.0,0.0,0.0,0.588,0.135,0.221,0.057,0.0,0.000,0.0,0.0
2023-06-30,0.000,0.353,0.212,0.208,0.109,0.000,0.0,0.0,0.0,0.0,0.593,0.133,0.219,0.055,0.0,0.000,0.0,0.0
2023-05-31,0.000,0.382,0.192,0.178,0.070,0.003,0.0,0.0,0.0,0.0,0.593,0.131,0.219,0.054,0.0,0.003,0.0,0.0


In [70]:
sheetname = "H Risk per YC"
df = pd.read_excel(file_path,sheet_name = sheetname, index_col=0)

df_Risk_YC_P = FindCol(df, "MD c. (PTF)_", "Portfolio ")
df_Risk_YC_B= FindCol(df, "MD c. (REF)_", "Benchmark ")

df_Risk_YC = pd.concat([df_Risk_YC_P, df_Risk_YC_B], axis=1)
df_Risk_YC.head()


,Portfolio Total,Portfolio USD,Portfolio EUR,Portfolio JPY,Portfolio GBP,Portfolio AUD,Portfolio NZD,Portfolio MXN,Portfolio NOK,Portfolio DKK,...,Benchmark JPY,Benchmark GBP,Benchmark AUD,Benchmark NZD,Benchmark MXN,Benchmark NOK,Benchmark DKK,Benchmark Other,Benchmark CAD,Benchmark PLN
2023-09-30,625,267,167,98,69,15,11,6,0,0,...,169,48,9,0,0,0,2,13,0,0
2023-08-31,0,262,163,99,51,15,10,6,0,0,...,169,50,9,0,0,0,0,0,12,0
2023-07-31,0,268,166,103,52,15,11,6,0,0,...,173,52,10,0,0,0,0,0,12,0
2023-06-30,0,245,161,108,47,15,5,6,1,0,...,175,51,10,0,0,0,0,0,13,0
2023-05-31,0,262,127,118,37,20,7,8,1,0,...,179,50,10,0,0,0,0,0,12,0


In [102]:
sheetname = "H Sector DTS"
df = pd.read_excel(file_path,sheet_name = sheetname, index_col=0)

df_Financials_expo = FindCol(df, "Expo weight (PTF)_Financials", "Financials Expo")/100
df_Financials_dts = FindCol(df, "DTS c. (PTF)_Financials", "Financials DTS")

df_Industrials_expo = FindCol(df, "Expo weight (PTF)_Industrials", "non Financials Expo")/100
df_Industrials_dts = FindCol(df, "DTS c. (PTF)_Industrials", "non Financials DTS")

df_Utilities_expo = FindCol(df, "Expo weight (PTF)_Utilities", "non Financials Expo")/100
df_Utilities_dts = FindCol(df, "DTS c. (PTF)_Utilities", "non Financials DTS")

df_Securitized_expo = FindCol(df, "Expo weight (PTF)_Securitized", "non Financials Expo")/100
df_Securitized_dts = FindCol(df, "DTS c. (PTF)_Securitized", "non Financials DTS")

df_nonFinancials_expo = df_Industrials_expo.add(df_Utilities_expo.add(df_Securitized_expo))
df_nonFinancials_dts = df_Industrials_dts.add(df_Utilities_dts.add(df_Securitized_dts))


df_expo = df_Financials_expo.add(df_nonFinancials_expo, fill_value = 0)
df_dts = df_Financials_dts.add(df_nonFinancials_dts, fill_value = 0)


,Financials DTS,non Financials DTS
2023-09-30,1.90,0.55
2023-08-31,1.88,0.64
2023-07-31,1.95,0.61
2023-06-30,2.16,0.64
2023-05-31,2.33,0.67
2023-04-30,2.44,0.70
2023-03-31,2.50,0.78
2023-02-28,1.96,0.72
2023-01-31,1.95,0.76
2022-12-31,2.12,0.81
